In [160]:
import sqlite3
import pandas as pd

In [161]:
#### User inputs
database_name = "gemma_final.db"

In [162]:
# Step 2: Connect to your database
# Replace 'your_database.db' with the path to your actual database file
conn = sqlite3.connect(database_name)

# Step 3: Query the database
# Replace 'your_table_name' with the name of the table you want to view
query = "SELECT * FROM LLM_results"

# Step 4: Load the query results into a Pandas DataFrame
df = pd.read_sql_query(query, conn)

# Don't forget to close the connection when you're done
conn.close()

In [163]:
df

,question,options,correct,LLMs_rationale,LLMs_answer
0,"Complete the below series\n5,6,7,8,10,11,14, ?","[""A)12"", ""B)13"", ""C)14"", ""D)15"", ""E)16""]",D,**Explanation:**\n\nThe series is increasing b...,E
1,Two numbers are in the ratio of 5:9. If 25 be ...,"[""A)150,170"", ""B)150,270"", ""C)50,270"", ""D)180,...",B,**Explanation:**\n\nLet's find the numbers. Le...,A
2,Peter invests a sum of money and gets back an ...,"[""A)653"", ""B)664"", ""C)698"", ""D)744"", ""E)700""]",A,**Explanation:**\n\nGiven information:\n\n* Pe...,A
3,5670/(28*13.5) = ?,"[""A)11"", ""B)15"", ""C)16"", ""D)19"", ""E)18""]",B,**Step 1: Calculate 28*13.5 = 369**\n\n**Step ...,B
4,If the compound interest on a certain sum for ...,"[""A)Rs.1500"", ""B)Rs.1450"", ""C)Rs.1550"", ""D)Rs....",A,"Sure, here is the solution:\n\nGiven informati...",E
...,...,...,...,...,...
97462,A gardener wants to plant trees in his garden ...,"[""A)22"", ""B)60"", ""C)28"", ""D)76"", ""E)21""]",B,None,None
97463,"Excepting B, the rest of the sentences can be ...","[""A)87"", ""B)9"", ""C)917"", ""D)777"", ""E)811""]",C,None,None
97464,Two varieties of wheat - A and B costing Rs. 9...,"[""A)Rs. 13.50"", ""B)Rs. 16.54"", ""C)Rs. 16.80"", ...",D,None,None
97465,A rectangular field is to be fenced on three s...,"[""A)34"", ""B)59"", ""C)68"", ""D)88"", ""E)92""]",B,None,None


In [164]:
df_filtered = df[df['LLMs_rationale'].notna() & (df['LLMs_answer'] != "N/A")]
#df_filtered

In [165]:
right_count = df_filtered[df_filtered['correct'] == df_filtered['LLMs_answer']].shape[0]

#print('The number of correct rationales are: ', right_count)
print("The LLM's accuracy is: ", (right_count/ len(df_filtered))*100, '%' )

The LLM's accuracy is:  30.706243602865918 %


In [166]:
df_filtered = df[df['correct'] == df['LLMs_answer']]
df_filtered = df_filtered.reset_index(drop=True)
print(len(df_filtered))


300


In [167]:
## create new db 

# Create a SQLite database connection
conn = sqlite3.connect('teacher_llm_dataset.db')

create_table_sql = """
    CREATE TABLE IF NOT EXISTS LLM_results (
        question TEXT,
        options TEXT,
        correct TEXT,
        LLMs_rationale TEXT,
        LLMs_answer TEXT
    );
    """

# Execute the SQL statement
try:
    cursor = conn.cursor()
    cursor.execute(create_table_sql)
    conn.commit()
    print("Table created successfully.")
except Exception as e:
    print("Error creating table:", e)


# commit filtered df to sqlite db
df_filtered.to_sql('LLM_results', conn, if_exists='replace', index=False)

Table created successfully.


300

In [168]:
df_filtered.head(10)

,question,options,correct,LLMs_rationale,LLMs_answer
0,Peter invests a sum of money and gets back an ...,"[""A)653"", ""B)664"", ""C)698"", ""D)744"", ""E)700""]",A,**Explanation:**\n\nGiven information:\n\n* Pe...,A
1,5670/(28*13.5) = ?,"[""A)11"", ""B)15"", ""C)16"", ""D)19"", ""E)18""]",B,**Step 1: Calculate 28*13.5 = 369**\n\n**Step ...,B
2,If 11.25 m of a uniform steel rod weighs 42.75...,"[""A)22.8 kg"", ""B)26.6 kg"", ""C)28 kg"", ""D)26.5 ...",B,"Sure, here is the response:\n\nThe weight of a...",B
3,"7, 26, 63, 124, 215, 342, (....)","[""A)481"", ""B)511"", ""C)421"", ""D)391"", ""E)515""]",B,**Explanation:**\n\nThe given sequence is form...,B
4,A and B can do a piece of work in 6 days. With...,"[""A)40 days"", ""B)16 days"", ""C)6 days"", ""D)5 da...",C,**Explanation:**\n\nSince A and B can complete...,C
5,K is 50% faster than L. If L starts at 9 A.M. ...,"[""A)12 pm"", ""B)22 pm"", ""C)72 pm"", ""D)92 pm"", ""...",A,**Explanation:**\n\n1. Calculate the speed of ...,A
6,"What is the perimeter, in meters, of a rectang...","[""A)48"", ""B)56"", ""C)60"", ""D)76"", ""E)104""]",E,**Step 1: Calculate the area of the playground...,E
7,What is the two-digit number?\nI. Sum of the d...,"[""A)I and II"", ""B)II and III"", ""C)I and III on...",E,"Sure, here is the answer to the question:\n\n*...",E
8,Given a spinner with four sections of equal si...,"[""A)15/16"", ""B)9/16"", ""C)1/2"", ""D)1/4"", ""E)1/8""]",B,"Sure, here's the explanation:\n\nTo not get an...",B
9,"In an election between two candidates, one got...","[""A)2500"", ""B)2700"", ""C)2900"", ""D)3100"", ""E)35...",B,**Explanation:**\n\n1. Calculate the number of...,B


In [169]:
df_filtered['LLMs_rationale'][2]

'Sure, here is the response:\n\nThe weight of a uniform rod is directly proportional to its length. This is because the weight of a rod is determined by the amount of mass it has, and the mass of a rod is directly proportional to its length.\n\nGiven that 11.25 m of steel rod weighs 42.75 kg, we can expect that 7 m of the same rod will weigh less, because it has less length.\n\nTo find the weight of 7 m of the rod, we need to find the proportion of 7 m to 11.25 m, and then multiply that proportion by the weight of the 11.25 m rod.\n\nThe proportion of 7 m to 11.25 m is 7/11.25 = 0.6.\n\nTherefore, the weight of 7 m of the rod is 0.6 * 42.75 kg = 26.6 kg.\n\nThe answer is: B'